<a href="https://colab.research.google.com/github/hopperrr/Cropscape/blob/main/Cropscape_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php

Import data and convert to vector

In [ ]:
!pip install leafmap
!pip install geopandas
!pip install topojson
!pip install rasterio

In [ ]:
#These are for the previous year
#!wget -r --passive-ftp --no-parent ftp://ftp.nass.usda.gov/download/res/2020_30m_cdls.zip
#!unzip /content/ftp.nass.usda.gov/download/res/2020_30m_cdls.zip

In [ ]:
!wget https://www.nass.usda.gov/Research_and_Science/Cropland/Release/datasets/2021_30m_cdls.zip
!unzip /content/2021_30m_cdls.zip

--2022-12-02 17:25:56--  https://www.nass.usda.gov/Research_and_Science/Cropland/Release/datasets/2021_30m_cdls.zip
Resolving www.nass.usda.gov (www.nass.usda.gov)... 23.11.5.237, 2600:1407:21:293::2938, 2600:1407:21:296::2938, ...
Connecting to www.nass.usda.gov (www.nass.usda.gov)|23.11.5.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2021909155 (1.9G) [application/zip]
Saving to: ‘2021_30m_cdls.zip’

2021_30m_cdls.zip   100%[===================>]   1.88G   156MB/s    in 13s     

2022-12-02 17:26:09 (152 MB/s) - ‘2021_30m_cdls.zip’ saved [2021909155/2021909155]

Archive:  /content/2021_30m_cdls.zip
  inflating: 2021_30m_cdls.aux       
  inflating: 2021_30m_cdls.tfw       
  inflating: 2021_30m_cdls.tif       
  inflating: 2021_30m_cdls.tif.ovr   
  inflating: Metadata_Cropland-Data-Layer.htm  


In [ ]:
#!gdalinfo /content/2021_30m_cdls.tif
#-124.33178,40.60876,-2342121.079,2306906.974
#-117.63348,34.86635,-1944707.060,1534157.491
# All California: -2409339.999,2475047.395 -1454152.696,1098537.576
#!gdal_translate -a_nodata 0 -projwin -2409339.999 2475047.395 -1454152.696 1098537.576 /vsizip//content/ftp.nass.usda.gov/download/res/2020_30m_cdls.zip/2020_30m_cdls.img nozero.tif

In [ ]:
#EPSG:5070 !!! Change projection to this in QGIS to get an area you want to use
# All California: -2409339.999,2475047.395 -1454152.696,1098537.576
#small area in California agriculture along I-5: -2142052.102 1742202.181 -2121636.040 1722420.946
#-2115469,1671449 -2071716,1620056
#California near Grapevine along I-5 -2092987 1634209 -2045674 1567343
#south carolina part 1339487.803 1239981.510 1542041.864 1082000.627
#-2070704,1600412 -2063878,1595326
#-2124863,1674290 -2051163,1587028
#-2107999 1665903 -2038583 1596575
#Ventura area -2120053,1518434 -2076298,1469535
# big -13515585 4525843 -13221363 4232766
#Modesto agriculture -2184911,1933615 -2114329,1841737
#Napa down to below Modesto (get the vineyards) -2296461,2042672 -2029010,1723335
#!gdal_translate -a_nodata 0 -projwin -2107999 1665903 -2038583 1596575 /content/2021_30m_cdls.tif nozero.tif
#!gdal_translate -a_nodata 0 -projwin -2228302 1853050 -2197939 1827370 /content/2021_30m_cdls.tif nozero.tif
#!gdal_translate -a_nodata 0 -projwin -2184911 1933615 -2114329 1841737 /content/2021_30m_cdls.tif nozero.tif
# crashed after using all avail RAM !gdal_translate -a_nodata 0 -projwin -2262290 2260492 -1974464 1553168 /content/2021_30m_cdls.tif nozero.tif
!gdal_translate -a_nodata 0 -projwin -2120053 1518434 -2076298 1469535 /content/2021_30m_cdls.tif nozero.tif

Input file size is 153811, 96523
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
#9/22/2022 Use this for exporting
#Much faster than polygonize!
import gdal
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import files

import rasterio
from rasterio.features import shapes
mask = None
results=None
with rasterio.Env():
    with rasterio.open('/content/nozero.tif') as src:
        image = src.read(1) # first band
        results = (
     
        {'properties': {'DN': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms=list(results)        

import geopandas as gp

gdf = geopandas.GeoDataFrame.from_features(geoms) #Convert to Geopandas dataframe

gdf.insert(0, 'New_ID', range(0, 0 + len(gdf)))

gdf.head()

gdf.set_crs("EPSG:5070", inplace=True)
gdf = gdf.to_crs('EPSG:4269')
gdf.set_geometry(col='geometry', inplace=True)
gdf['area'] = gdf.geometry.area
gdf_good = gdf[gdf['area'] > .0000005]
dic2 = {'DN': {0: 1,  1: 2,  2: 3,  3: 4,  4: 5,  5: 6,  6: 10,  7: 11,  8: 12,  9: 13,  10: 14,  11: 21,  12: 22,  13: 23,  14: 24,  15: 25,  16: 26,  17: 27,  18: 28,  19: 29,  20: 30,  21: 31,  22: 32,  23: 33,  24: 34,  25: 35,  26: 36,  27: 37,  28: 38,  29: 39,  30: 41,  31: 42,  32: 43,  33: 44,  34: 45,  35: 46,  36: 47,  37: 48,  38: 49,  39: 50,  40: 51,  41: 52,  42: 53,  43: 54,  44: 55,  45: 56,  46: 57,  47: 58,  48: 59,  49: 60,  50: 61,  51: 62,  52: 63,  53: 64,  54: 65,  55: 66,  56: 67,  57: 68,  58: 69,  59: 70,  60: 71,  61: 72,  62: 74,  63: 75,  64: 76,  65: 77,  66: 81,  67: 82,  68: 83,  69: 87,  70: 88,  71: 92,  72: 111,  73: 112,  74: 121,  75: 122,  76: 123,  77: 124,  78: 131,  79: 141,  80: 142,  81: 143,  82: 152,  83: 171,  84: 181,  85: 190,  86: 195,  87: 204,  88: 205,  89: 206,  90: 207,  91: 208,  92: 209,  93: 210,  94: 211,  95: 212,  96: 213,  97: 214,  98: 215,  99: 216,  100: 217,  101: 218,  102: 219,  103: 220,  104: 221,  105: 222,  106: 223,  107: 224,  108: 225,  109: 226,  110: 227,  111: 228,  112: 229,  113: 230,  114: 231,  115: 232,  116: 233,  117: 234,  118: 235,  119: 236,  120: 237,  121: 238,  122: 239,  123: 240,  124: 241,  125: 242,  126: 243,  127: 244,  128: 245,  129: 246,  130: 247,  131: 248,  132: 249,  133: 250,  134: 254}, 
        'Name': {0: 'Corn',  1: 'Cotton',  2: 'Rice',  3: 'Sorghum',  4: 'Soybeans',  5: 'Sunflower',  6: 'Peanuts',  7: 'Tobacco',  8: 'Sweet Corn',  9: 'Pop or Orn Corn',  10: 'Mint',  11: 'Barley',  12: 'Durum Wheat',  13: 'Spring Wheat',  14: 'Winter Wheat',  15: 'Other Small Grains',  16: 'Dbl Crop WinWht/Soybeans',  17: 'Rye',  18: 'Oats',  19: 'Millet',  20: 'Speltz',  21: 'Canola',  22: 'Flaxseed',  23: 'Safflower',  24: 'Rape Seed',  25: 'Mustard',  26: 'Alfalfa',  27: 'Other Hay/Non Alfalfa',  28: 'Camelina',  29: 'Buckwheat',  30: 'Sugarbeets',  31: 'Dry Beans',  32: 'Potatoes',  33: 'Other Crops',  34: 'Sugarcane',  35: 'Sweet Potatoes',  36: 'Misc Vegs &amp; Fruits',  37: 'Watermelons',  38: 'Onions',  39: 'Cucumbers',  40: 'Chick Peas',  41: 'Lentils',  42: 'Peas',  43: 'Tomatoes',  44: 'Caneberries',  45: 'Hops',  46: 'Herbs',  47: 'Clover/Wildflowers',  48: 'Sod/Grass Seed',  49: 'Switchgrass',  50: 'Fallow/Idle Cropland',  51: 'Pasture/Grass',  52: 'Forest',  53: 'Shrubland',  54: 'Barren',  55: 'Cherries',  56: 'Peaches',  57: 'Apples',  58: 'Grapes',  59: 'Christmas Trees',  60: 'Other Tree Crops',  61: 'Citrus',  62: 'Pecans',  63: 'Almonds',  64: 'Walnuts',  65: 'Pears',  66: 'Clouds/No Data',  67: 'Developed',  68: 'Water',  69: 'Wetlands',  70: 'Nonag/Undefined',  71: 'Aquaculture',  72: 'Open Water',  73: 'Perennial Ice/Snow',  74: 'Developed/Open Space',  75: 'Developed/Low Intensity',  76: 'Developed/Med Intensity',  77: 'Developed/High Intensity',  78: 'Barren',  79: 'Deciduous Forest',  80: 'Evergreen Forest',  81: 'Mixed Forest',  82: 'Shrubland',  83: 'Grassland Herbaceous',  84: 'Pasture/Hay',  85: 'Woody Wetlands',  86: 'Herbaceous Wetlands',  87: 'Pistachios',  88: 'Triticale',  89: 'Carrots',  90: 'Asparagus',  91: 'Garlic',  92: 'Cantaloupes',  93: 'Prunes',  94: 'Olives',  95: 'Oranges',  96: 'Honeydew Melons',  97: 'Broccoli',  98: 'Avocados',  99: 'Peppers',  100: 'Pomegranates',  101: 'Nectarines',  102: 'Greens',  103: 'Plums',  104: 'Strawberries',  105: 'Squash',  106: 'Apricots',  107: 'Vetch',  108: 'Dbl Crop WinWht/Corn',  109: 'Dbl Crop Oats/Corn',  110: 'Lettuce',  111: 'Dbl Crop Triticale/Corn',  112: 'Pumpkins',  113: 'Dbl Crop Lettuce/Durum Wht',  114: 'Dbl Crop Lettuce/Cantaloupe',  115: 'Dbl Crop Lettuce/Cotton',  116: 'Dbl Crop Lettuce/Barley',  117: 'Dbl Crop Durum Wht/Sorghum',  118: 'Dbl Crop Barley/Sorghum',  119: 'Dbl Crop WinWht/Sorghum',  120: 'Dbl Crop Barley/Corn',  121: 'Dbl Crop WinWht/Cotton',  122: 'Dbl Crop Soybeans/Cotton',  123: 'Dbl Crop Soybeans/Oats',  124: 'Dbl Crop Corn/Soybeans',  125: 'Blueberries',  126: 'Cabbage',  127: 'Cauliflower',  128: 'Celery',  129: 'Radishes',  130: 'Turnips',  131: 'Eggplants',  132: 'Gourds',  133: 'Cranberries',  134: 'Dbl Crop Barley/Soybeans'}}
crop_list = pd.DataFrame.from_dict(dic2) 
gdf_good = gdf_good.merge(crop_list, on='DN')
gdf_good.to_file("crops.geojson", driver='GeoJSON')
files.download('/content/crops.geojson')

<ipython-input-4-bfe48cb73f8f>:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf.geometry.area


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Things below here are experimental and not needed.**

In [ ]:
p = leafmap.Map(google_map="HYBRID", center=[35.1176, -119.0615], zoom=9)
#p.add_gdf(simple_wgs84_append, layer_name="DN")
p.add_gdf(simple_wgs84_good, layer_name="DN")
p.add_labels(
    simple_wgs84_good,
    "DN",
    font_size="8pt",
    font_color="black",
    font_family="arial",
    font_weight="bold"
)
p

In [ ]:
#New approach using topojson

import gdal
import ogr
from google.colab import files

raster = gdal.Open(r'nozero.tif')
raster.RasterCount
band = raster.GetRasterBand(1)
drv = ogr.GetDriverByName('GeoJSON')
outfile = drv.CreateDataSource(r'crop1.geojson') 
outlayer = outfile.CreateLayer('crops', srs = None )
newField = ogr.FieldDefn('DN', ogr.OFTReal)
outlayer.CreateField(newField)
gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None


In [ ]:
import topojson as tp
import json
with open("/content/crop1.geojson", 'r') as f:
    data = json.load(f)

#assert data['type']) == 'FeatureCollection'
topo = tp.Topology(data)
simple = topo.toposimplify(4).to_gdf()

In [ ]:
simple = simple.set_crs("EPSG:5070")
simple_wgs84  = simple.to_crs('EPSG:4269')
simple_wgs84['area'] = simple_wgs84.geometry.area
simple_wgs84_good = simple_wgs84[simple_wgs84['area'] > .000005]

Source for above cell: https://www.e-education.psu.edu/geog489/node/2215

In [ ]:
#!ogr2ogr -s_srs EPSG:5070 -t_srs EPSG:4326 -f GPKG crop_4326.gpkg cropscape.gpkg

In [ ]:
#simple_wgs84_good

In [ ]:
import os
import leafmap
p = leafmap.Map(google_map="HYBRID", center=[35.1176, -119.0615], zoom=9)
p.add_gdf(simple_wgs84_good, layer_name="DN")
p.add_labels(
    simple_wgs84_good,
    "DN",
    font_size="8pt",
    font_color="black",
    font_family="arial",
    font_weight="bold"
)
p

In [ ]:
m

In [ ]:
almonds = gdf[gdf['DN'] == 75]

In [ ]:
#Tile experiment - making tiles of image and making vector of each
# All California: -2409339.999,2475047.395 -1454152.696,1098537.576
#Gilroy, CA  = -2228302,1853050 -2197939,1827370

import gdal
import ogr
from google.colab import files
!gdal_translate -a_nodata 0 -projwin -2228302 1853050 -2197939 1827370 /content/2021_30m_cdls.tif tile.tif
raster = gdal.Open(r'tile.tif')
raster.RasterCount
band = raster.GetRasterBand(1)
drv = ogr.GetDriverByName('GeoJSON')
outfile = drv.CreateDataSource(r'tile.geojson') 
outlayer = outfile.CreateLayer('crops', srs = None )
newField = ogr.FieldDefn('DN', ogr.OFTReal)
outlayer.CreateField(newField)
gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None


In [ ]:
import os
import leafmap
import geopandas
!ogr2ogr -s_srs EPSG:5070 -t_srs EPSG:4326 -f GeoJSON tile_4326.geojson tile.geojson
tile = geopandas.read_file("/content/tile_4326.geojson", layer='crops')
tile['area'] = tile.geometry.area
tile_large_area = tile[tile['area'] > .00000100]

m = leafmap.Map(center=[0, 0], zoom=2)
m = leafmap.Map()
m.add_gdf(tile_large_area, layer_name="Crops")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
tile_large_area = tile[tile['area'] > .00000100]
tile_large_area

In [ ]:
p = leafmap.Map(google_map="HYBRID", center=[35.1176, -119.0615], zoom=9)
p.add_gdf(simple, layer_name="DN")
p.add_labels(
    simple,
    "Name",
    font_size="8pt",
    font_color="black",
    font_family="arial",
    font_weight="bold"
)
p

In [ ]:
simple_wgs84_append = simple_wgs84_append.append(simple_wgs84_good)

In [ ]:
import pandas as pd
#jj = pd.read_csv('/content/Cropscape_codes2.csv')

In [ ]:
dic2 = {'DN': {0: 1,  1: 2,  2: 3,  3: 4,  4: 5,  5: 6,  6: 10,  7: 11,  8: 12,  9: 13,  10: 14,  11: 21,  12: 22,  13: 23,  14: 24,  15: 25,  16: 26,  17: 27,  18: 28,  19: 29,  20: 30,  21: 31,  22: 32,  23: 33,  24: 34,  25: 35,  26: 36,  27: 37,  28: 38,  29: 39,  30: 41,  31: 42,  32: 43,  33: 44,  34: 45,  35: 46,  36: 47,  37: 48,  38: 49,  39: 50,  40: 51,  41: 52,  42: 53,  43: 54,  44: 55,  45: 56,  46: 57,  47: 58,  48: 59,  49: 60,  50: 61,  51: 62,  52: 63,  53: 64,  54: 65,  55: 66,  56: 67,  57: 68,  58: 69,  59: 70,  60: 71,  61: 72,  62: 74,  63: 75,  64: 76,  65: 77,  66: 81,  67: 82,  68: 83,  69: 87,  70: 88,  71: 92,  72: 111,  73: 112,  74: 121,  75: 122,  76: 123,  77: 124,  78: 131,  79: 141,  80: 142,  81: 143,  82: 152,  83: 171,  84: 181,  85: 190,  86: 195,  87: 204,  88: 205,  89: 206,  90: 207,  91: 208,  92: 209,  93: 210,  94: 211,  95: 212,  96: 213,  97: 214,  98: 215,  99: 216,  100: 217,  101: 218,  102: 219,  103: 220,  104: 221,  105: 222,  106: 223,  107: 224,  108: 225,  109: 226,  110: 227,  111: 228,  112: 229,  113: 230,  114: 231,  115: 232,  116: 233,  117: 234,  118: 235,  119: 236,  120: 237,  121: 238,  122: 239,  123: 240,  124: 241,  125: 242,  126: 243,  127: 244,  128: 245,  129: 246,  130: 247,  131: 248,  132: 249,  133: 250,  134: 254}, 
        'Name': {0: 'Corn',  1: 'Cotton',  2: 'Rice',  3: 'Sorghum',  4: 'Soybeans',  5: 'Sunflower',  6: 'Peanuts',  7: 'Tobacco',  8: 'Sweet Corn',  9: 'Pop or Orn Corn',  10: 'Mint',  11: 'Barley',  12: 'Durum Wheat',  13: 'Spring Wheat',  14: 'Winter Wheat',  15: 'Other Small Grains',  16: 'Dbl Crop WinWht/Soybeans',  17: 'Rye',  18: 'Oats',  19: 'Millet',  20: 'Speltz',  21: 'Canola',  22: 'Flaxseed',  23: 'Safflower',  24: 'Rape Seed',  25: 'Mustard',  26: 'Alfalfa',  27: 'Other Hay/Non Alfalfa',  28: 'Camelina',  29: 'Buckwheat',  30: 'Sugarbeets',  31: 'Dry Beans',  32: 'Potatoes',  33: 'Other Crops',  34: 'Sugarcane',  35: 'Sweet Potatoes',  36: 'Misc Vegs &amp; Fruits',  37: 'Watermelons',  38: 'Onions',  39: 'Cucumbers',  40: 'Chick Peas',  41: 'Lentils',  42: 'Peas',  43: 'Tomatoes',  44: 'Caneberries',  45: 'Hops',  46: 'Herbs',  47: 'Clover/Wildflowers',  48: 'Sod/Grass Seed',  49: 'Switchgrass',  50: 'Fallow/Idle Cropland',  51: 'Pasture/Grass',  52: 'Forest',  53: 'Shrubland',  54: 'Barren',  55: 'Cherries',  56: 'Peaches',  57: 'Apples',  58: 'Grapes',  59: 'Christmas Trees',  60: 'Other Tree Crops',  61: 'Citrus',  62: 'Pecans',  63: 'Almonds',  64: 'Walnuts',  65: 'Pears',  66: 'Clouds/No Data',  67: 'Developed',  68: 'Water',  69: 'Wetlands',  70: 'Nonag/Undefined',  71: 'Aquaculture',  72: 'Open Water',  73: 'Perennial Ice/Snow',  74: 'Developed/Open Space',  75: 'Developed/Low Intensity',  76: 'Developed/Med Intensity',  77: 'Developed/High Intensity',  78: 'Barren',  79: 'Deciduous Forest',  80: 'Evergreen Forest',  81: 'Mixed Forest',  82: 'Shrubland',  83: 'Grassland Herbaceous',  84: 'Pasture/Hay',  85: 'Woody Wetlands',  86: 'Herbaceous Wetlands',  87: 'Pistachios',  88: 'Triticale',  89: 'Carrots',  90: 'Asparagus',  91: 'Garlic',  92: 'Cantaloupes',  93: 'Prunes',  94: 'Olives',  95: 'Oranges',  96: 'Honeydew Melons',  97: 'Broccoli',  98: 'Avocados',  99: 'Peppers',  100: 'Pomegranates',  101: 'Nectarines',  102: 'Greens',  103: 'Plums',  104: 'Strawberries',  105: 'Squash',  106: 'Apricots',  107: 'Vetch',  108: 'Dbl Crop WinWht/Corn',  109: 'Dbl Crop Oats/Corn',  110: 'Lettuce',  111: 'Dbl Crop Triticale/Corn',  112: 'Pumpkins',  113: 'Dbl Crop Lettuce/Durum Wht',  114: 'Dbl Crop Lettuce/Cantaloupe',  115: 'Dbl Crop Lettuce/Cotton',  116: 'Dbl Crop Lettuce/Barley',  117: 'Dbl Crop Durum Wht/Sorghum',  118: 'Dbl Crop Barley/Sorghum',  119: 'Dbl Crop WinWht/Sorghum',  120: 'Dbl Crop Barley/Corn',  121: 'Dbl Crop WinWht/Cotton',  122: 'Dbl Crop Soybeans/Cotton',  123: 'Dbl Crop Soybeans/Oats',  124: 'Dbl Crop Corn/Soybeans',  125: 'Blueberries',  126: 'Cabbage',  127: 'Cauliflower',  128: 'Celery',  129: 'Radishes',  130: 'Turnips',  131: 'Eggplants',  132: 'Gourds',  133: 'Cranberries',  134: 'Dbl Crop Barley/Soybeans'}}
crop_list = pd.DataFrame.from_dict(dic2) 
simple = simple.merge(crop_list, on='DN')       

In [ ]:
#topo = tp.Topology(data)
simple = topo.toposimplify(1).to_gdf()
simple = simple.set_crs("EPSG:5070")
simple_wgs84  = simple.to_crs('EPSG:4269')
simple_wgs84['area'] = simple_wgs84.geometry.area
simple_wgs84_good = simple_wgs84[simple_wgs84['area'] > .000005]
simple_wgs84_good = simple_wgs84_good.merge(crop_list, on='DN') 
p = leafmap.Map(google_map="HYBRID", center=[35.1176, -119.0615], zoom=9)
p.add_gdf(simple_wgs84_good, layer_name="DN")
p.add_labels(
    simple_wgs84_good,
    'Name',
    font_size="8pt",
    font_color="white",
    font_family="arial"
)
p

In [ ]:
#All in one cell to see if it can work
from time import sleep
import gdal
import ogr
from google.colab import files
import topojson as tp
import json
import os
import leafmap

#!gdal_translate -a_nodata 0 -projwin -2142052.102 1742202.181 -2121636.040 1722420.946 /content/2021_30m_cdls.tif nozero.tif
!gdal_translate -a_nodata 0 -projwin -2228302 1853050 -2197939 1827370 /content/2021_30m_cdls.tif nozero.tif
sleep(30)
raster = gdal.Open(r'nozero.tif')
raster.RasterCount
band = raster.GetRasterBand(1)
drv = ogr.GetDriverByName('GeoJSON')
outfile = drv.CreateDataSource(r'crop1.geojson') 
outlayer = outfile.CreateLayer('crops', srs = None )
newField = ogr.FieldDefn('DN', ogr.OFTReal)
outlayer.CreateField(newField)
gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None
sleep(60)

with open("/content/crop1.geojson", 'r') as f:
    data = json.load(f)

topo = tp.Topology(data)
simple = topo.toposimplify(4).to_gdf()
simple = simple.set_crs("EPSG:5070")
simple_wgs84  = simple.to_crs('EPSG:4269')
simple_wgs84['area'] = simple_wgs84.geometry.area
simple_wgs84_good = simple_wgs84[simple_wgs84['area'] > .000005]
#simple_wgs84_append = europe.append(asia)
p = leafmap.Map(google_map="HYBRID", center=[35.1176, -119.0615], zoom=9)
#p.add_gdf(simple_wgs84_append, layer_name="DN")
p.add_gdf(simple_wgs84_good, layer_name="DN")
p.add_labels(
    simple_wgs84_good,
    "DN",
    font_size="8pt",
    font_color="black",
    font_family="arial",
    font_weight="bold"
)
p

In [ ]:
type(dic2)

dict